# Training a Baseline Model: GPT-2 fine-tuned for NLI using all of SNLI train data

### 1. Imports and Global Settings

In [1]:
from datasets import load_dataset, disable_caching
from transformers import GPT2ForSequenceClassification, GPT2TokenizerFast, DataCollatorWithPadding, set_seed
import torch
from torch.nn.functional import one_hot
import sys
sys.path.append('..')
from utils_ import tokenize, train_classifier
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
set_seed(42)
disable_caching()

### 2. Pre-Processing
- Get SNLI Dataset (Train fold)
- Remove instances without gold standard labels, i.e., label = -1
- One-hot encoding for labels
- Tokenise data

In [2]:
train = load_dataset('snli', split = 'train')
train = train.filter(lambda x: x['label'] != -1).map( \
    lambda x: {'label': one_hot(torch.tensor(x['label']), 3).type(torch.float32).numpy()}, \
    batched = True)

Reusing dataset snli (/home/shana92/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)


  0%|          | 0/551 [00:00<?, ?ba/s]

  0%|          | 0/550 [00:00<?, ?ba/s]

In [3]:
# set up tokeniser
# padding to left because GPT2 uses last token for prediction
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2-medium", padding_side = 'left', \
                                              padding = True, truncation = True)
tokenizer.pad_token = tokenizer.eos_token # pad with 'eos' token

In [4]:
# tokenize data
train = train.map(lambda x: tokenize(tokenizer, x['premise'] + '|' + x['hypothesis']))
len_bef_exclusion = len(train)

# exclude instances with > 128 tokens
train = train.filter(lambda x: x['exclude'] == False)
len_aft_exclusion = len(train)

# print message if instances were in fact excluded
if len_bef_exclusion - len_aft_exclusion > 0:
    
    print(f'{len_bef_exclusion - len_aft_exclusion} ' + \
          f'({(len_bef_exclusion/len_aft_exclusion - 1)*100:>2f}%) sequences excluded')

  0%|          | 0/549367 [00:00<?, ?ex/s]

  0%|          | 0/550 [00:00<?, ?ba/s]

In [5]:
# keep only needed columns, set data format to PyTorch
train.set_format(type = 'torch', columns = ['label', 'input_ids', 'attention_mask'])

### 3. Model

In [6]:
# set up data collator - https://huggingface.co/docs/transformers/main_classes/data_collator
# this is a (callable) helper object that sends batches of data to the model
data_collator = DataCollatorWithPadding(tokenizer, padding = 'max_length', \
                                         return_tensors = 'pt', max_length = 128)

In [7]:
# hyper-parameters for model training
batch_size = 32 # constrained by memory
lr = 1e-5 # set to match Le et al. (2020) - https://arxiv.org/abs/2002.04108

In [8]:
# set up a dataloader (batch generator)
dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size, \
                                 shuffle=True, collate_fn=data_collator) # batch size constrained by GPU memory

In [9]:
# instantiate GPT2 classifier model
model = GPT2ForSequenceClassification.from_pretrained("gpt2-medium", 
                                  num_labels=3,
                                  problem_type="multi_label_classification")
model.config.pad_token_id = model.config.eos_token_id # specify pad_token used by tokenizer

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2-medium and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# set up optimizer (loss function in-built)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
# Train and Save model
torch.save(train_classifier(model, dataloader, optimizer, device), 'baseline_unfiltered.pth')

  0%|          | 0/17168 [00:00<?, ?it/s]

loss: 1.106129  [    0/549367]
loss: 0.652977  [54912/549367]
loss: 0.355182  [109824/549367]
loss: 0.413212  [164736/549367]
loss: 0.357015  [219648/549367]
loss: 0.478685  [274560/549367]
loss: 0.202242  [329472/549367]
loss: 0.176116  [384384/549367]
loss: 0.287832  [439296/549367]
loss: 0.239158  [494208/549367]
loss: 0.237825  [549120/549367]
Epoch average loss: 0.34859874844551086


  0%|          | 0/17168 [00:00<?, ?it/s]

loss: 0.340378  [    0/549367]
loss: 0.214370  [54912/549367]
loss: 0.231678  [109824/549367]
loss: 0.188228  [164736/549367]
loss: 0.343081  [219648/549367]
loss: 0.146512  [274560/549367]
loss: 0.167774  [329472/549367]
loss: 0.148926  [384384/549367]
loss: 0.230051  [439296/549367]
loss: 0.280149  [494208/549367]
loss: 0.192146  [549120/549367]
Epoch average loss: 0.22900183498859406


  0%|          | 0/17168 [00:00<?, ?it/s]

loss: 0.162329  [    0/549367]
loss: 0.092864  [54912/549367]
loss: 0.227731  [109824/549367]
loss: 0.202142  [164736/549367]
loss: 0.270213  [219648/549367]
loss: 0.200030  [274560/549367]
